<a href="https://colab.research.google.com/github/NirantK/Hinglish/blob/mergeall/misc/MajorityVoting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
pred_output_path = Path("/content/drive/My Drive/HinglishNLP/PredOutput")

In [ ]:
l = pd.read_csv("test_labels_hinglish.txt")
ref = pd.read_csv("/content/drive/My Drive/BERT_valid64.csv")

DistilBert1 = pd.read_csv(pred_output_path / "D1-test-full-output.csv")
DistilBert2 = pd.read_csv(pred_output_path / "D2-test-full-output.csv")
DistilBert3 = pd.read_csv(pred_output_path / "D3-test-full-output.csv")
Bert1 = pd.read_csv(pred_output_path / "B1-test-full-output.csv")
Bert2 = pd.read_csv(pred_output_path / "B2-test-full-output.csv")

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(ref["actual"])
clfs = [DistilBert1, DistilBert2, DistilBert3, Bert1, Bert2]
for df in clfs:
    df["y"] = le.transform(df["Sentiment"])

# Majority Voting | the _how_?
Modified version of code from [this blog](https://sebastianraschka.com/Articles/2014_ensemble_classifier.html) about Majority Voting.

- `np.bincount(classes_[:,0])` -> returns bin-count for each prediction class for a given output. So if out of five classifiers one said positive[0] and one said neutral[1] and three said negative[2] the output would look like array([1, 1, 3])
- `np.argmax` -> get the pred which has the max bin-count.

In [ ]:
classes_ = np.asarray([clf["y"] for clf in clfs])
maj = np.asarray(
    [np.argmax(np.bincount(classes_[:, c])) for c in range(classes_.shape[1])]
)

In [ ]:
print(
    precision_recall_fscore_support(maj, le.transform(ref["actual"]), average="macro")
)
print(accuracy_score(maj, le.transform(ref["actual"])))

In [ ]:
DistilBert1_final = pd.read_csv(pred_output_path / "D1-final_test-full-output.csv")
DistilBert2_final = pd.read_csv(pred_output_path / "D2-final_test-full-output.csv")
DistilBert3_final = pd.read_csv(pred_output_path / "D3-final_test-full-output.csv")
Bert1_final = pd.read_csv(pred_output_path / "B1-final_test-full-output.csv")
Bert2_final = pd.read_csv(pred_output_path / "B2-final_test-full-output.csv")

In [ ]:
clfs = [
    DistilBert1_final,
    DistilBert2_final,
    DistilBert3_final,
    Bert1_final,
    Bert2_final,
]
for df in clfs:
    df["y"] = le.transform(df["Sentiment"])

In [ ]:
classes_ = np.asarray([clf["y"] for clf in clfs])
maj = np.asarray(
    [np.argmax(np.bincount(classes_[:, c])) for c in range(classes_.shape[1])]
)

In [ ]:
print(
    precision_recall_fscore_support(
        maj, le.transform(l["Sentiment"][:-1]), average="macro"
    )
)
print(accuracy_score(maj, le.transform(l["Sentiment"][:-1])))